In [ ]:
import pandas as pd

from summer2 import CompartmentalModel
from summer2.parameters import Parameter, DerivedOutput

In [ ]:
seir_data = pd.read_csv(
    "./csvs/model_4_3a.csv",
    header=53,
    index_col=0,
    nrows=18251,
    usecols=range(5),
)

In [ ]:
seir_data

In [ ]:
def build_seir_model(
    config: dict,
) -> CompartmentalModel:

    # Model characteristics
    compartments = ("Susceptible", "Pre-infectious", "Infectious", "Immune")
    model = CompartmentalModel(
        times=(0, config["end_time"]),
        compartments=compartments,
        infectious_compartments=["Infectious"],
        timestep=config["t_step"],
    )
    model.set_initial_population(
        distribution={
            "Susceptible": config["tot_popn"] - config["infous_0"],
            "Infectious": config["infous_0"],
        }
    )
    
    # Transitions
    r0 = Parameter("r0")
    ave_infous = Parameter("ave_infous")
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=r0 / ave_infous,
        source="Susceptible", 
        dest="Pre-infectious",
    )
    model.add_transition_flow(
        name="progression", 
        fractional_rate=1. / Parameter("ave_preinfous"),
        source="Pre-infectious", 
        dest="Infectious",
    )
    model.add_transition_flow(
        name="recovery", 
        fractional_rate=1. / ave_infous,
        source="Infectious", 
        dest="Immune",
    )
    model.add_universal_death_flows(
        "universal_death",
        death_rate=1. / Parameter("life_expectancy") / 365.,
    )
    model.add_replacement_birth_flow(
        "births",
        "Susceptible",
    )

    # Outputs
    model.request_output_for_flow(
        name="incidence", 
        flow_name="progression"
    )
    model.request_output_for_compartments(
        name="total_population",
        compartments=compartments,
        save_results=False,
    )
    model.request_output_for_compartments(
        name="susceptible",
        compartments=("Susceptible",),
        save_results=False,
    )
    model.request_output_for_compartments(
        name="immune",
        compartments=("Immune",),
        save_results=False,
    )
    total_population = DerivedOutput("total_population")
    model.request_function_output(
        name="susc_prop",
        func=DerivedOutput("susceptible") / total_population,
    )
    model.request_function_output(
        name="immune_prop",
        func=DerivedOutput("immune") / total_population,
    )    
    model.request_function_output(
        name="r_n",
        func=DerivedOutput("susc_prop") * r0,
    )
    return model

In [ ]:
model_config = {
    "tot_popn": 1e5,
    "infous_0": 1.,
    "end_time": 18250.,
    "t_step": 1.,
    "prop_immune_0": 0.3,
}

parameters = {
    "r0": 13.,
    "ave_preinfous": 8.,
    "ave_infous": 7.,
    "life_expectancy": 70.,
}

In [ ]:
seir_demog_model = build_seir_model(model_config)
seir_demog_model.run(parameters=parameters, solver="euler")
seir_values = seir_demog_model.get_outputs_df()

In [ ]:
assert abs(seir_data - seir_values).max().max() < 1e5

In [ ]:
fig, (left_ax, right_ax) = plt.subplots(1, 2, figsize=(14, 5))

times = seir_demog_model.times / 365.
left_ax.plot(times, derived_outputs["immune_prop"], color="lime", label="Immune")
left_ax.set_xlabel("Time (days)")
left_ax.xaxis.set_minor_locator(AutoMinorLocator(4))
left_ax.set_ylim(0.88, 0.96)
left_ax.set_yticks(np.linspace(0.88, 0.96, 3))
left_ax.set_ylabel("Proportion immune")
left_ax.legend()
left_ax.spines.top.set_visible(False)
left_ax.set_title("Figure 3")

twin_ax = left_ax.twinx()
twin_ax.plot(times, derived_outputs["incidence"], color="magenta", label="Incidence")
twin_ax.set_xlim(40, 50)
twin_ax.set_yticks(range(0, 25, 5))
twin_ax.set_ylim(0, 20)
twin_ax.set_ylabel("Number of new infectious persons/day")
twin_ax.legend(loc=4)
twin_ax.spines.top.set_visible(False)

image = plt.imread("./figures/model_4_3a_fig_3.JPG")
right_ax.axis("off")
right_ax.imshow(image, aspect="auto");